In [1]:
import pandas as pd
import snowflake.connector
from collections import defaultdict
import json
import re

In [2]:
con = snowflake.connector.connect(user='vishal.kumar@scale.com',
                                 account='pxa65918',
                                 authenticator='externalbrowser',
                                 warehouse='COMPUTE_WH',
                                 database='SCALE_CRAWLER',
                                 role='GENERAL_RO')

cs = con.cursor()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [3]:
sql = '''
SELECT 
RESULT
FROM PUBLIC.SPOTTERAUDITS   
WHERE AUDIT_TYPE = 'CustomerFeedback' and CUSTOMER_FEEDBACK_DATE_SENT='2022-08-12' AND RESULT IS NOT NULL
'''
cs.execute(sql)

df = cs.fetch_pandas_all()
df
df.to_csv('alltoolingdata.csv')

In [4]:
main_df = {}
for i in range(len(df)):
    df_x = json.loads(df['RESULT'][i])

#    if df_x == "":
#        continue

main_df.update(df_x)

main_df
#csv_df = pd.DataFrame.from_dict(main_df,orient='index')
#csv_df
#csv_df.to_csv (r'test9.csv', index = True, header=True)



{'2309602739108969': {'additional_image_link': {'result': 'Fully Correct by the Customer',
   'valueIsBlank': False},
  'brand': {'result': 'Fully Correct by the Customer', 'valueIsBlank': False},
  'color': {'result': 'Non-Reproducible', 'valueIsBlank': True},
  'description': {'result': 'Fully Correct by the Customer',
   'valueIsBlank': False},
  'image_link': {'result': 'Fully Correct by the Customer',
   'valueIsBlank': False},
  'material': {'result': 'Fully Correct by the Customer',
   'valueIsBlank': False},
  'size': {'comment': '',
   'reason': 'Irrelevant',
   'result': 'Reproducible',
   'valueIsBlank': False},
  'title': {'result': 'Fully Correct by the Customer', 'valueIsBlank': False}},
 '2402463469819018': {'additional_image_link': {'result': 'Fully Correct by the Customer',
   'valueIsBlank': False},
  'brand': {'result': 'Fully Correct by the Customer', 'valueIsBlank': False},
  'color': {'result': 'Fully Correct by the Customer', 'valueIsBlank': False},
  'descriptio

In [5]:
#print(type(main_df))

def media_score_finder(test_res):
    media_score = [] 
    if 'image_link' in test_res:
        if test_res['image_link']['result'] == 'Reproducible' or test_res['additional_image_link']['result'] == 'Reproducible':
            media_score.append('Reproducible')
            if test_res['image_link']['result'] == 'Reproducible':
                media_score.append(test_res['image_link']['reason'])
            else:
                media_score.append(test_res['additional_image_link']['reason'])
        elif test_res['image_link']['result'] == "Can't Audit" or test_res['additional_image_link']['result'] == "Can't Audit":
            media_score = "Can't Audit"
        elif test_res['image_link']['result'] == 'Needs Clarification' or test_res['additional_image_link']['result'] == 'Needs Clarification':
            media_score = 'Needs Clarification'
        elif test_res['image_link']['result'] == 'Fully Correct by the Customer' and test_res['additional_image_link']['result'] == 'Fully Correct by the Customer':
            media_score = 'Fully Correct by the Customer'
        else:
             media_score = 'Non-Reproducible'
    return media_score


In [6]:
def variant_score(variant_result):

    if 'image_link' in variant_result:
        media_score = media_score_finder(variant_result) 
    else: 
        return "Can't Audit"
    
    if 'description' in variant_result:
        description_score = variant_result['description']['result']
    else:
        return "Can't Audit"
    
   
    if 'title' in variant_result:
        title_score = variant_result['title']['result']   
    else:
        return "Can't Audit"
   
    if media_score[0] == 'Reproducible':
        variant_result['media_error'] = True
        variant_result['media_error_reason'] =  media_score[1]
    else:
        variant_result['media_error'] = False
    
    if description_score  == 'Reproducible':
        variant_result['description_error'] = True
    else:
        variant_result['description_error'] = False
    
    if title_score == 'Reproducible':
        variant_result['title_error'] = True
    else:
        variant_result['title_error'] = False
        
    
    variant_score = ""
    
    if media_score == 'Reproducible' or description_score == 'Reproducible' or title_score == 'Reproducible':
        variant_score = 'Reproducible'
        variant_result['final_score'] = 'Reproducible'
    elif media_score == "Can't Audit" or description_score == "Can't Audit" or title_score == "Can't Audit":
        variant_score = "Can't Audit"
        variant_result['final_score'] = "Can't Audit"
    elif media_score == 'Needs Clarification' or description_score == 'Needs Clarification' or title_score == 'Needs Clarification':
        variant_score = 'Needs Clarification'
        variant_result['final_score'] = 'Needs Clarification'
    elif  media_score == 'Fully Correct by the Customer' and description_score == 'Fully Correct by the Customer' or title_score == 'Fully Correct by the Customer':
        variant_score = 'Fully Correct by the Customer'
        variant_result['final_score'] =  'Fully Correct by the Customer'
    else:
        variant_score = 'Non-Reproducible'
        
    return variant_score
        

In [7]:
def get_scores(result):
    f = x = c = n = y = 0

    counter = 0
    
    error_list = {}
    for key in result:
        counter+=1
        vs = variant_score(result[key])

        if vs == 'Fully Correct by the Customer':
            f+=1
        elif vs == "Can't Audit":
            c+=1
        elif vs == 'Non-Reproducible':
            n+=1
        elif vs == 'Needs Clarification':
            x+=1
        elif vs == 'Reproducible':
            y+=1
            error_list[key] = []
            if result[key]['description_error']:
                error_list[key].append(('description_error',result[key]['description']['reason']))
            if result[key]['title_error']:
                error_list[key].append(('title_error',result[key]['title']['reason']))
            if  result[key]['media_error']:
                 error_list[key].append(('media_error',result[key]['media_error_reason']))

    total_variants = f + x + c + n + y 
    variant_pass_rate = round((f+n)/total_variants,2)
    error_rate = round(y/total_variants,2)
    nc = round(x/total_variants,3)
    ca =  round(c/total_variants,3)
  
    print("Total Variants: ", total_variants)
    print("Variant Pass Rate: ",variant_pass_rate)
    print("Variant Error Rate: ",error_rate)
    print("Needs Clarification: ",nc)
    print("Can't Audit: ",ca)
    return error_list

In [8]:
error_list = get_scores(main_df)

Total Variants:  31
Variant Pass Rate:  1.0
Variant Error Rate:  0.0
Needs Clarification:  0.0
Can't Audit:  0.0


In [ ]:
def error_report(error_list):
    total_error_count = description_error = media_error = title_error =  0
    
    part_missing = hanging_sentence = formatting = links = other = mismatch = purchase_logistics = CTA = broken_link = 0
    
    for key in error_list:
        for i in range(len(error_list[key])):
            if error_list[key][i][0] == 'description_error':
                description_error +=1
                if error_list[key][i][1] =='Part Missing':
                    part_missing+=1
                elif error_list[key][i][1] =='Other':
                    other+=1
                elif error_list[key][i][1] == 'CTA':
                    CTA+=1
                elif error_list[key][i][1] == 'Hanging Sentence':
                    hanging_sentence+=1
                elif error_list[key][i][1] == 'Broken Link':
                    broken_link+=1
                elif error_list[key][i][1] == 'Formatting':
                    formatting+=1
                elif error_list[key][i][1] == 'Mismatch':
                    mismatch+=1
                elif error_list[key][i][1] == 'Other Purchase Logistics ':
                    purchase_logistics+=1
                elif error_list[key][i][1] == 'Links':
                    links+=1
            elif error_list[key][i][0] == 'media_error':
                media_error += 1
            elif error_list[key][i][0] == 'title_error':
                title_error += 1

    total_error_count = media_error + title_error + description_error
   
    print("Total Errors: ", total_error_count)
    print(description_error)
    print("Description errors as  % of total Errors: ",(round(description_error/total_error_count,2)))
    print(media_error)
    print("Media errors as % of total Errors: ",(round(media_error/total_error_count,2)))
    print(title_error)
    print("Title errors as % of total Errors: ",(round(title_error/total_error_count,2)))
     
        
    print("The top error bucket was Description - Links ",(round(links/total_error_count,2)))

In [ ]:
error_report(error_list)

In [ ]:
def get_catalog_explain_links(error_list):
    for key in error_list:
        error_list[key].append("https://dashboard.crawler.scale.com/catalog-explain?q="+str(key))

In [ ]:
get_catalog_explain_links(error_list)

In [ ]:
error_list

In [ ]:
feedback_df = pd.DataFrame.from_dict(main_df,orient='index')
main_df.update(error_list)

In [ ]:
#feedback_df

In [ ]:
#feedback_df.to_csv (r'test8.csv', index = False, header=True)

def get_media_score(row):
    if row['image_link']['result'] == 'Reproducible' or row['additional_image_link']['result'] == 'Reproducible':
        return 'Reproducible'
    elif row['image_link']['result'] == "Can't Audit" or row['additional_image_link']['result'] == "Can't Audit":
        return "Can't Audit"
    elif row['image_link']['result'] == 'Needs Clarification' or row['additional_image_link']['result'] == 'Needs Clarification':
        return 'Needs Clarification'
    elif row['image_link']['result'] == 'Fully Correct by the Customer' and row['additional_image_link']['result'] == 'Fully Correct by the Customer':
        return  'Fully Correct by the Customer'
    else:
        return "Non-Reproducible"


In [ ]:
def get_media_reason(row):
    if row['image_link']['result'] == 'Reproducible':
        return row['image_link']['reason']
    elif row['additional_image_link']['result'] == 'Reproducible':
        return row['additional_image_link']['reason']
    else:
        return 'No Media Error'

In [ ]:
def get_description_score(row):
    return row['description']['result']

def get_description_reason(row):
    if row['description_score'] == 'Reproducible':
        return row['description']['reason']
    else:
        return "No Description Error"

def get_title_score(row):
    return row['title']['result']

def get_title_reason(row):
    if row['title_score'] == 'Reproducible':
        return row['title']['reason']
    else:
        return "No Title Error"

In [ ]:
feedback_df['media_score'] =  feedback_df.apply(lambda row: get_media_score(row), axis=1)
feedback_df['media_error_reason'] =  feedback_df.apply(lambda row: get_media_reason(row), axis=1)
feedback_df['description_score'] =  feedback_df.apply(lambda row: get_description_score(row), axis=1)
feedback_df['description_error_reason'] =  feedback_df.apply(lambda row: get_description_reason(row), axis=1)
feedback_df['title_score'] =  feedback_df.apply(lambda row: get_title_score(row), axis=1)
feedback_df['title_error_reason'] =  feedback_df.apply(lambda row: get_title_reason(row), axis=1)

In [ ]:
def get_variant_score(row):
    media_score = row['media_score']
    description_score = row['description_score']
    title_score = row['title_score']
    
    
    if media_score == 'Reproducible' or description_score == 'Reproducible' or title_score == 'Reproducible':
        return 'Reproducible'
    elif media_score == "Can't Audit" or description_score == "Can't Audit" or title_score == "Can't Audit":
        return "Can't Audit"
    elif media_score == 'Needs Clarification' or description_score == 'Needs Clarification' or title_score == 'Needs Clarification':
        return 'Needs Clarification'
    elif  media_score == 'Fully Correct by the Customer' and description_score == 'Fully Correct by the Customer' or title_score == 'Fully Correct by the Customer':
        return  'Fully Correct by the Customer'
    else:
        return'Non-Reproducible'

In [ ]:
def get_catalog_explain_link(row):
    return "https://dashboard.crawler.scale.com/catalog-explain?q="+str(row.index)

In [ ]:
feedback_df['final_score'] =  feedback_df.apply(lambda row: get_variant_score(row), axis=1)

In [ ]:
feedback_df['link'] = feedback_df.apply(lambda row: get_catalog_explain_link(row), axis=1)

In [ ]:
feedback_df['count'] = 1

In [ ]:
grp_df = feedback_df.groupby('final_score').agg({'count':'sum'})
grp_df['percent'] = round((grp_df['count'] / grp_df['count'].sum()) * 100,0)
grp_df

In [ ]:
grp_df = feedback_df.groupby('media_score').agg({'count':'sum'})
grp_df['percent'] = round((grp_df['count'] / grp_df['count'].sum()) * 100,0)
grp_df

In [ ]:
grp_df = feedback_df.groupby('title_score').agg({'count':'sum'})
grp_df['percent'] = round((grp_df['count'] / grp_df['count'].sum()) * 100,0)
grp_df

In [ ]:
grp_df = feedback_df.groupby('description_score').agg({'count':'sum'})
grp_df['percent'] = round((grp_df['count'] / grp_df['count'].sum()) * 100,0)
grp_df

In [ ]:
grp_df = feedback_df.where(feedback_df['media_error_reason'] !='No Media Error').groupby('media_error_reason').agg({'count':'sum'})
grp_df['percent'] = round((grp_df['count'] / grp_df['count'].sum()) * 100,0)
grp_df

In [ ]:
Media_errors